In [42]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer 

Es el momento de eliminar los nulos:

Usaremos el data frame de Limpieza III.

 - Reemplazad los valores nulos de la columna age por la media de la edad usando el método SimpleImputer.

In [15]:
df = pd.read_csv("../../datos/attacks_limpio_3.csv", index_col = 0)

In [16]:
imputer = SimpleImputer(strategy= "mean", missing_values = np.nan)

In [17]:
imputer = imputer.fit(df[["age"]])

In [18]:
df["age"] = imputer.transform(df[["age"]])

In [19]:
# Hemos sobreescrito la columna 'age' reemplazando los valores nulos por por la media
df.isnull().sum()

case_number       1
year              0
mes             576
country          50
type              4
area            456
activity        544
age               0
sex             569
species        5196
injury           28
fatal           619
dtype: int64

 - Reemplazad los valores nulos de la columna sex por la moda, usando el método SimpleImputer.

In [20]:
imputer = SimpleImputer(strategy= "most_frequent", missing_values = np.nan)

In [21]:
imputer = imputer.fit(df[["sex"]])

In [22]:
df["sex"] = imputer.transform(df[["sex"]])

In [23]:
df.isnull().sum()

case_number       1
year              0
mes             576
country          50
type              4
area            456
activity        544
age               0
sex               0
species        5196
injury           28
fatal           619
dtype: int64

 - Reemplazad los valores nulos de la columna type por el valor más frecuente (la moda) con el método SimpleImputer.

In [24]:
imputer = SimpleImputer(strategy= "most_frequent", missing_values = np.nan)

In [25]:
imputer = imputer.fit(df[["type"]])

In [26]:
df["type"] = imputer.transform(df[["type"]])

In [27]:
df.isnull().sum()

case_number       1
year              0
mes             576
country          50
type              0
area            456
activity        544
age               0
sex               0
species        5196
injury           28
fatal           619
dtype: int64

- Utilizad el método KNN Imputer para reemplazar todos los valores nulos de las columnas numéricas.

In [29]:
df2 = pd.read_csv('../../datos/attacks_limpio_3.csv', index_col=0)

In [31]:
df2.head(2)

,case_number,year,mes,country,type,area,activity,age,sex,species,injury,fatal
0,1800.00.00,1976,NaN,seychelles,Unprovoked,St. Anne,a corsair's boat was overturned,NaN,F,NaN,"FATAL, all onboard were killed by sharks",y
1,1797.05.28.R,1976,May,NaN,Unprovoked,NaN,Dropped overboard,NaN,NaN,NaN,FATAL,y


In [32]:
df2.isnull().sum()

case_number       1
year              0
mes             576
country          50
type              4
area            456
activity        544
age            2873
sex             569
species        5196
injury           28
fatal           619
dtype: int64

In [34]:
numericas = df2.select_dtypes(include = np.number)
imputerKNN = KNNImputer(n_neighbors = 5)
imputerKNN.fit(numericas)

KNNImputer()

In [35]:
numericas_knn= imputerKNN.transform(numericas)

In [36]:
df_knn_imputer = pd.DataFrame(numericas_knn, columns = numericas.columns)

In [38]:
columnas_knn = df_knn_imputer.columns
columnas_knn

Index(['year', 'age'], dtype='object')

In [39]:

df.drop(columnas_knn, axis = 1, inplace = True)

In [40]:
df[columnas_knn] = numericas_knn

In [41]:
df.isnull().sum()

case_number       1
mes             576
country          50
type              0
area            456
activity        544
sex               0
species        5196
injury           28
fatal           619
year              0
age               0
dtype: int64


 - Utilizad el método Iterative Imputer para reemplazar todos los valores nulos de las columnas numéricas.
 

In [72]:
df3 = pd.read_csv('../../datos/attacks_limpio_3.csv', index_col=0)

In [73]:
df3.isnull().sum()

case_number       1
year              0
mes             576
country          50
type              4
area            456
activity        544
age            2873
sex             569
species        5196
injury           28
fatal           619
dtype: int64

In [74]:
numericas = df3.select_dtypes(include = np.number)

In [75]:
imputer = IterativeImputer(n_nearest_features=None, imputation_order='ascending') 

In [76]:
imputer.fit(numericas)

IterativeImputer()

In [77]:
imputer.transform(numericas)

array([[1976.        ,   26.51519845],
       [1976.        ,   26.51519845],
       [1976.        ,   26.51519845],
       ...,
       [1976.        ,   26.51519845],
       [1976.        ,   26.51519845],
       [1976.        ,   26.51519845]])

In [78]:
numericas_trans = pd.DataFrame(imputer.transform(numericas), columns = numericas.columns) 

In [79]:
numericas_trans.isnull().sum()

year    0
age     0
dtype: int64

In [80]:
columnas = numericas_trans.columns 

In [81]:
df3.drop(columnas, axis = 1, inplace = True) 

In [82]:
df3[columnas] = numericas_trans[columnas]

In [83]:
df3.columns

Index(['case_number', 'mes', 'country', 'type', 'area', 'activity', 'sex',
       'species', 'injury', 'fatal', 'year', 'age'],
      dtype='object')

In [84]:
df3.isnull().sum() 

case_number       1
mes             576
country          50
type              4
area            456
activity        544
sex             569
species        5196
injury           28
fatal           619
year              0
age               0
dtype: int64

-- Cuál es la diferencia entre los dos últimos métodos? --

La diferencia entre el IterativeImputer y el KNNImputer es que mientras el II infiere los valores faltantes a partir de una regresión utilizando el resto de las variables, el KNNI infiere los valores nulos analizando los valores de la variable faltante en base a los valores más cercanos al mismo. Es por ello que en función de la cantidad de vecinos que decidamos utilizar, el valor predicho cambiará. Para poder decir cuál es el mejor valor para mi número de vecinos, se podrían utilizar varios diferentes y hacer un histograma de los datos con los valores ya reemplazados. El número de vecinos que nos haya dado una distribución más similar a la original sería el más adecuado para utilizar. 

2.Guardad el csv para seguir trabajando con el en los siguientes ejercicios de pair

In [85]:
# Guardamos el data frame donde teniamos tanto las variables categóricas como las variables numéricas
# reemplazadas por la moda y la media respectivamente

df.to_csv("../../datos/attacks_limpieza_5.csv")